### 1) 라이브러리 설치

In [6]:
%pip install -q sentence-transformers langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

#### OpenAIEmbeddings 예제

In [2]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩할 텍스트 데이터 (난이도 높은 문장)
text = "심층 신경망을 활용한 자연어 처리는 문맥을 이해하고 텍스트 생성을 가능하게 하는 강력한 기술로 자리 잡았습니다."

# 1. 쿼리(단일 텍스트)를 임베딩
query_result = embeddings.embed_query(text)
print(query_result[:5])  # 임베딩 벡터의 처음 5개 요소 출력

# 2. 문서(여러 개의 텍스트)를 임베딩
texts = [text, text, text, text]  # 동일한 문장을 4번 사용

doc_result = embeddings.embed_documents(texts)
print(len(doc_result))  # 생성된 문서 벡터 개수 출력
print(doc_result[0][:5])  # 첫 번째 문서 벡터의 처음 5개 요소 출력

# 3. 1024차원의 임베딩 생성
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
print(len(embeddings_1024.embed_documents([text])[0]))  # 벡터 차원 확인


[0.008827518671751022, 0.06394866108894348, 0.027438532561063766, 0.027418192476034164, -0.0031374304089695215]
4
[0.008827518671751022, 0.06394866108894348, 0.027438532561063766, 0.027418192476034164, -0.0031374304089695215]
1024


In [3]:
# 4. 임베딩 벡터를 활용한 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

# 비교할 문장 5개
sentence1 = "자연어 처리는 기계가 인간 언어를 이해하고 생성할 수 있도록 돕는 기술입니다."
sentence2 = "심층 학습을 적용한 언어 모델은 다양한 문맥을 학습하여 정교한 문장을 생성할 수 있습니다."
sentence3 = "통계적 자연어 처리 기법과 신경망 모델은 언어 이해에 중요한 역할을 합니다."
sentence4 = "Neural networks have significantly improved the accuracy of machine translation."
sentence5 = "안녕하세요? 만나서 반가워요."

sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
print(len(sentences), len(sentences[0]))

embedded_sentences = embeddings_1024.embed_documents(sentences)

def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

# 각 문장 간 유사도 비교
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}")

5 43
[유사도 0.3926] 자연어 처리는 기계가 인간 언어를 이해하고 생성할 수 있도록 돕는 기술입니다. 	 <=====> 	 심층 학습을 적용한 언어 모델은 다양한 문맥을 학습하여 정교한 문장을 생성할 수 있습니다.
[유사도 0.5590] 자연어 처리는 기계가 인간 언어를 이해하고 생성할 수 있도록 돕는 기술입니다. 	 <=====> 	 통계적 자연어 처리 기법과 신경망 모델은 언어 이해에 중요한 역할을 합니다.
[유사도 0.3639] 자연어 처리는 기계가 인간 언어를 이해하고 생성할 수 있도록 돕는 기술입니다. 	 <=====> 	 Neural networks have significantly improved the accuracy of machine translation.
[유사도 0.1976] 자연어 처리는 기계가 인간 언어를 이해하고 생성할 수 있도록 돕는 기술입니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4779] 심층 학습을 적용한 언어 모델은 다양한 문맥을 학습하여 정교한 문장을 생성할 수 있습니다. 	 <=====> 	 통계적 자연어 처리 기법과 신경망 모델은 언어 이해에 중요한 역할을 합니다.
[유사도 0.3284] 심층 학습을 적용한 언어 모델은 다양한 문맥을 학습하여 정교한 문장을 생성할 수 있습니다. 	 <=====> 	 Neural networks have significantly improved the accuracy of machine translation.
[유사도 0.2276] 심층 학습을 적용한 언어 모델은 다양한 문맥을 학습하여 정교한 문장을 생성할 수 있습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4358] 통계적 자연어 처리 기법과 신경망 모델은 언어 이해에 중요한 역할을 합니다. 	 <=====> 	 Neural networks have significantly improved the accuracy of machine translation.
[유사도 0.12

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

# Hugging Face 임베딩 모델 사용
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 단일 문장 임베딩
query_text = "자연어 처리는 기계가 인간 언어를 이해하는 기술입니다."
query_embedding = embeddings.embed_query(query_text)

# 여러 문장 임베딩
texts = [
    "인공지능은 다양한 산업에서 활용됩니다.",
    "자연어 처리는 언어를 분석하는 기술입니다.",
    "딥러닝을 활용한 모델이 성능이 좋습니다."
]
doc_embeddings = embeddings.embed_documents(texts)

# 결과 확인
print(query_embedding[:5])  # 쿼리 임베딩 벡터 일부 출력
print(len(doc_embeddings))  # 문서 벡터 개수 출력


[-0.02077852003276348, 0.06788308918476105, 0.11902482062578201, -0.03419821336865425, -0.07493330538272858]
3
